In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import numpy as np
import pandas as pd 

from sklearn.metrics import f1_score, accuracy_score

In [2]:
df = pd.read_csv('/kaggle/input/dataset-nlpvin/fixed_data.csv')

In [3]:
text = [txt[2:-2] for txt in df['combine_text']] 
label = df['label'].values

In [4]:
train_text, test_text, train_label, test_label = train_test_split(text, label, test_size=0.1, random_state=42) 
train_text, val_text, train_label, val_label = train_test_split(train_text, train_label, test_size=0.1, random_state=42)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")

if tokenizer.bos_token is None:
    tokenizer.bos_token = '<bos>'
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids('<bos>')

if tokenizer.eos_token is None:
    tokenizer.eos_token = '<eos>'
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('<eos>')

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
print(len(train_text))
print(len(train_label))

print(len(test_text))
print(len(test_label))

print(len(val_text))
print(len(val_label))

3053
3053
377
377
340
340


In [7]:
class SentimentDataset(Dataset): 
  def __init__(self, text, label, tokenizer, max_length=512): 
    self.text = text
    self.label = label 
    self.tokenizer = tokenizer 
    self.max_length = max_length

  def __len__(self): 
    return len(self.label) 

  def __getitem__(self, idx): 
    text_embedding = self.tokenizer.encode_plus(self.text[idx], return_tensors='pt', padding='max_length', max_length=self.max_length, truncation=True)
    label = torch.tensor(self.label[idx], dtype=torch.long) 
    return text_embedding['input_ids'].squeeze(), label
  

train_dataset = SentimentDataset(train_text, train_label, tokenizer)
test_dataset = SentimentDataset(test_text, test_label, tokenizer)
val_dataset = SentimentDataset(val_text, val_label, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [8]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        output = self.W_o(self.combine_heads(attn_output))
        return output

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [9]:
class SentimentTransformer(nn.Module): 
  def __init__(self, vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout): 
    super(SentimentTransformer, self).__init__() 
    self.vocab_size = vocab_size 
    self.d_model = d_model 
    self.num_heads = num_heads
    self.num_layers = num_layers 
    self.d_ff = d_ff 
    self.max_seq_length = max_seq_length 
    self.dropout = nn.Dropout(dropout) 

    self.embedding = nn.Embedding(vocab_size, d_model) 
    self.encoder = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

    self.positional_embedding = PositionalEncoding(d_model, max_seq_length) 

    self.fc = nn.Sequential(
        nn.Linear(max_seq_length * d_model, d_model), 
        nn.ReLU(), 
        nn.Linear(d_model, 128), 
        nn.ReLU(), 
        nn.Linear(128, 2)
    )

  def generate_mask(self, x): 
    x_mask = (x != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2).to(x.device) 
    return x_mask 

  def forward(self, x): 
    x_mask = self.generate_mask(x)

    x = self.embedding(x) * math.sqrt(self.d_model)
    x = self.dropout(self.positional_embedding(x)) 

    for layer in self.encoder: 
      x = layer(x, x_mask) 
    
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x) 

    return x
  
  
x = torch.randint(size=(128, 64), low=0, high=1000) 

net = SentimentTransformer(vocab_size=1000, d_model=512, num_heads=8, num_layers=4, d_ff=2048, max_seq_length=64, dropout=0.1) 

print(net(x).shape)

torch.Size([128, 2])


In [10]:
vocab_size = tokenizer.vocab_size 
d_model = 512 
num_heads = 8 
num_layers = 4 
d_ff = 2048 
max_seq_length = 512
dropout = 0.1 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

model = SentimentTransformer(vocab_size=vocab_size, d_model=d_model, num_heads=num_heads, num_layers=num_layers, d_ff=d_ff, max_seq_length=max_seq_length, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=1e-5) 

epochs = 50

train_loss = []
val_loss = []

for epoch in range(epochs):
  model.train()
  total_train_loss = 0
  for inputs, labels in tqdm(train_loader):

    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    total_train_loss += loss.item()
    train_loss.append(loss.item())

  print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {total_train_loss / len(train_loader)}')

  model.eval()
  total_val_loss = 0
  f_1 = 0
  accuracy = 0
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      total_val_loss += loss.item()

      val_loss.append(loss.item())

      preds = torch.argmax(outputs, dim=-1)
      f_1 += f1_score(labels.cpu(), preds.cpu())
      accuracy += accuracy_score(labels.cpu(), preds.cpu())

  print(f'Epoch {epoch + 1}/{epochs}, Val Loss: {total_val_loss / len(val_loader)}')
  print(f'F1 Score: {f_1 / len(val_loader)}')
  print(f'Accuracy: {accuracy / len(val_loader)}')

torch.save(model.state_dict(), '/kaggle/working/trans_sen.pth')

total_test_loss = 0
f_1_test = 0
accuracy_test = 0

for inputs, labels in tqdm(test_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)

        preds = torch.argmax(outputs, dim=-1)
        f_1_test += f1_score(labels.cpu(), preds.cpu())
        accuracy_test += accuracy_score(labels.cpu(), preds.cpu())

print(f'Test F1 Score: {f_1_test / len(test_loader)}')
print(f'Test Accuracy: {accuracy_test / len(test_loader)}')

100%|██████████| 191/191 [01:04<00:00,  2.98it/s]


Epoch 1/50, Train Loss: 0.39095523684280703


100%|██████████| 22/22 [00:02<00:00,  7.71it/s]


Epoch 1/50, Val Loss: 0.3274438137357885
F1 Score: 0.4068181818181818
Accuracy: 0.8522727272727273


100%|██████████| 191/191 [01:06<00:00,  2.86it/s]


...

Epoch 50/50, Train Loss: 0.011085649617955523


100%|██████████| 22/22 [00:02<00:00,  8.00it/s]


Epoch 50/50, Val Loss: 0.4883016883316238
F1 Score: 0.5161616161616162
Accuracy: 0.8920454545454546


 71%|███████   | 17/24 [00:02<00:00,  7.94it/s]/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|██████████| 24/24 [00:02<00:00,  8.02it/s]

Test F1 Score: 0.3992063492063492
Test Accuracy: 0.8807870370370371
